In [ ]:
import numpy as np
class EcoSystem():
    """EcoSystem - vanilla GA algorithm"""

    def __init__(self,
                 evaluator,
                 ecosystem_size, 
                 population_size, 
                 chromosome_sizes,
                 tournament_coef = 2,
                 crossover_coef = [1],
                 mutation_coef = [0.01],
                 elitism_coef = 0.05,
                 temp = 10):

        # Input tests
        assert(population_size % 2 == 0)
        
        # Creating EcoSystem
        self.EVALUATOR = evaluator
        self.ECOSYSTEM_SIZE = ecosystem_size
        self.POPULATION_SIZE = population_size
        self.CHROMOSOME_SIZES = chromosome_sizes
        self.TOURNAMENT_COEF = tournament_coef
        self.CROSSOVER_COEF = crossover_coef
        self.MUTATION_COEF = mutation_coef
        self.ELITISM_COEF = elitism_coef
        self.T0 = temp

    def _ecosystem_environment(self):
        """Preparing ecosystem environment"""

        self.ecosystem = []
        self.population_solutions = []
        self.ELITES_NUM = np.round(self.ELITISM_COEF * self.POPULATION_SIZE).astype(int)
        self.ENTROPY_POOL = {
            i : np.random.binomial(1, i, 500000)
            for i in set(self.CROSSOVER_COEF + self.MUTATION_COEF + [0.5])
        }

    def evolve(self, epochs):
        """Starting optimization cycle"""

        self._ecosystem_environment()

        # Imbueing EcoSystem with life        
        for i in range(self.ECOSYSTEM_SIZE):
            self.ecosystem.append(Population())

            # Copying algorithm's parameters to Population instances
            for j in self.__dict__.keys():
                if (j.isupper()):
                    setattr(self.ecosystem[-1], j, getattr(self, j))
            
            self.ecosystem[-1]._population_environment()
            self.ecosystem[-1].grow_population()

            for epoch in range(epochs):
                self.ecosystem[-1]._evaluate_population()
                self.ecosystem[-1]._acceptance()
                self.ecosystem[-1].annealing_schedule(epoch, epochs)
                self.ecosystem[-1].selection()
                self.ecosystem[-1].crossover()
                self.ecosystem[-1].mutation()
                self.ecosystem[-1].elite_preservation()
                
            self.ecosystem[-1]._evaluate_population()
            self.ecosystem[-1]._acceptance()
            self.population_solutions.append([
                    self.ecosystem[i].fittest_genome_score,
                    self.ecosystem[i].fittest_genome
            ])


class Population():
    """Class encapsulates population's information and behaviour"""

    def __init__(self):
        pass
    
    def _population_environment(self):
        """Preparing population environment"""

        self.T = self.T0
        self.genome = [None for i in self.CHROMOSOME_SIZES]
        self.evolved_genome = [None for i in self.CHROMOSOME_SIZES]
        self.fittest_genome = [None for i in self.CHROMOSOME_SIZES]
        self.elite_genome = [None for i in self.CHROMOSOME_SIZES]

    def grow_population(self):
        """Initializing population with random binary data"""

        for i in range(len(self.genome)):
            self.genome[i] = np.random.randint(0, 2, size = (self.POPULATION_SIZE, self.CHROMOSOME_SIZES[i]))
            self.evolved_genome[i] = np.random.randint(0, 2, size = (self.POPULATION_SIZE, self.CHROMOSOME_SIZES[i]))
            self.elite_genome[i] = np.empty((self.ELITES_NUM, self.CHROMOSOME_SIZES[i]))
        
        self.scores = self.EVALUATOR(self.genome)

    def _evaluate_population(self):
        """Population evaluation"""

        self.evolved_scores = self.EVALUATOR(self.evolved_genome)

    def _acceptance(self):
        """Annealing schedule - Accepting solutions based on probability derived from temperature"""

        acceptance_mask = np.logical_or(
            np.random.uniform(0, 1, self.POPULATION_SIZE) <\
                1.0 / (1 + np.exp((self.evolved_scores - self.scores) / (self.T + 0.1))),
            self.scores > self.evolved_scores
        )

        self.scores[acceptance_mask] = self.evolved_scores[acceptance_mask]
        
        fittest_genome_index = np.argmin(self.scores)
        self.fittest_genome_score = self.scores[fittest_genome_index]
        
        elite_indices = np.argpartition(
            self.scores,
            self.ELITES_NUM
        )[: self.ELITES_NUM]

        for i in range(len(self.genome)):
            self.genome[i][acceptance_mask, :] = self.evolved_genome[i][acceptance_mask, :]
            self.elite_genome[i] = np.take(self.genome[i], elite_indices, axis = 0)
            self.fittest_genome[i] = self.genome[i][fittest_genome_index, :]

    def annealing_schedule(self, epoch, epochs):
        """Annealing schedule implemented as linear cooling"""

        self.T -= self.T0 * 1.0 / epochs

    def selection(self):
        """Selection process implemented as n-member tournament"""

        tournament_selector = np.random.randint(
            0,
            self.POPULATION_SIZE,
            (self.TOURNAMENT_COEF, self.POPULATION_SIZE)
        )
        winner_selector_indices = np.argmin(np.take(self.scores, tournament_selector), axis = 0)
        winner_indices = np.choose(winner_selector_indices, tournament_selector)

        for i in range(len(self.genome)):            
            self.evolved_genome[i] = np.take(self.genome[i], winner_indices, axis = 0)

    def crossover(self):
        """Sexual recombination implemented as uniform crossover operator"""

        for i in range(len(self.genome)):

            index_mid = int(self.POPULATION_SIZE / 2)

            individual_selector = self._binomial(self.CROSSOVER_COEF[i], index_mid)
            gene_selector = self._binomial(
                0.5,
                (index_mid, self.CHROMOSOME_SIZES[i])
            )
            
            mask = np.bitwise_and(individual_selector[:, np.newaxis], gene_selector)

            recombination_mask = np.bitwise_and(
                np.bitwise_xor(self.evolved_genome[i][: index_mid, :], self.evolved_genome[i][index_mid :, :]),
                mask
            )

            self.evolved_genome[i][index_mid :, :], self.evolved_genome[i][: index_mid, :] =\
                np.bitwise_xor(self.evolved_genome[i][index_mid :, :], recombination_mask),\
                np.bitwise_xor(self.evolved_genome[i][: index_mid, :], recombination_mask)

    def mutation(self):
        """Mutation processs implemented as uniform mutation operator"""

        for i in range(len(self.genome)):
            mutation_selector = self._binomial(self.MUTATION_COEF[i], self.genome[i].shape)
            self.evolved_genome[i] = np.bitwise_xor(self.evolved_genome[i], mutation_selector)

    def elite_preservation(self):
        """Elites taking their rightful place"""

        for i in range(len(self.genome)):
            self.genome[i][: self.ELITES_NUM, :] = self.elite_genome[i]

    def _binomial(self, p, shape):
        """Provides recycled binomial distribution sequences"""

        if type(shape) is int:
            index = np.random.randint(0, 500000 - shape, 1)[0]
            return self.ENTROPY_POOL[p][index : index + shape]
        elif type(shape) is tuple:
            index = np.random.randint(0, 500000 - shape[0]*shape[1], 1)[0]
            return self.ENTROPY_POOL[p][index : index + shape[0]*shape[1]].reshape(shape)